# Model Server

The Intel Neural Compute Stick 2 is a compact and yet powerful Vision Processing Unit (VPU) which is optimized to perform the complex matrix calculations required for convolutional neural networks. Similar to a GPU, the VPU is engineered specifically for computer vision inference on images and videos.

The VPU is initalized with the network structure of the neural networks as well as the weights; these networks are then allocated space on the device. Total time to initialize a network is approximately 30 seconds, however once it has been initalized successive inference on a loaded model is processed in milliseconds. 

In order for multiple modules to make use of the VPU, it is therefor desirable for the device to be initialized once with multiple models. 

To accomplish this I have engineered the Model Server. The Model Server is responsable for initializing the VPU device, and registering the various Networks with the device. The Model Server then makes an Inference Service API available to the modules. If a module needs inference performed, it uses the Model Server API to provide the server with the kind of model needed, as well as the image requirements. The model server then uses its most recent image and performs inference on the VPU, returning the results to the model module where additional  processing can take place.

In order to receive the most recent images, the Modeling Server offers a second API endpoint for the Image Server Module. A limitation of ROS Noetic is that a service node cannot subscribe to other nodes like normal nodes can; this is because the service node needs to continually monitor the API requests. As such, the Model Server cannot obtain images on its own outside of an API call. In order to minimize the bandwidth and the duplication of images, the Image Server offers a kind of "reverse" service. Rather than acting as an API endpoint, the Image Server makes a service call to the Model Server requesting to give it an image via the Image Fetching Service offered by the model server. The Model Server then accepts that image, and the Image Server carries on with the management of the Image flags and requests for the nodes. 

In [3]:
from viper_toolkit import Dissect
import inspect

## The Model Server Module

In [33]:
from scripts import model_server
from model_server import ModelServer
source = inspect.getsource(ModelServer)
print (source)

ImportError: cannot import name 'ModelServer' from 'model_server' (/home/andrew/viper/devel/lib/python3/dist-packages/model_server/__init__.py)

In [5]:
from model_server import NeuralNetworkLoader
source = inspect.getsource(NeuralNetworkLoader)
print (source)

class NeuralNetworkLoader(object):
    """
    
    A NeuralNetworkLoader is a class object which loads a pretrained
    model (architecture and weights) onto a initialized OpenVino
    inference engine.
    
    Keyword Arguments:
    
    ie -- an Inference Engine instance set up in the parent node which 
    we will load this model to.
    
    ViperModel -- a instance of class ViperModel which contains the 
    models weights and the structure of the neural network.
    
    device -- the inference device to be used to predict on (i.e., 
    "MYRIAD", CPU, GPU, etc.)
    
    model_tag -- a three letter abbreviation used by the VIPER Logger
    module which identifies log messages as originating from within this
    modules code.
    
    model_name -- a logger attribute which identifies this model.
    
    """
    
    def __init__(self, 
                ie: IECore, 
                viper_model: ViperModel,
                device: str,
                model_tag: str = "M..",
    

In [6]:
from model_server import ViperModel
source = inspect.getsource(ViperModel)
print (source)

class ViperModel(object):
    # A ViperModel contains the location of the model architecture
    # and the model weights which are stored in the node package.
    
    def __init__(self, package_name, model_xml, weights_bin):
        self.pkg = package_name
        self.model = model_xml
        self.weights = weights_bin
        self.setup_model()
        
    def setup_model(self):
        self.dir = roslib.packages.get_pkg_dir(self.pkg)
        self.location = os.path.join(
            self.dir, 
            self.model
            )
        self.weights = os.path.join(
            self.dir, 
            self.weights
            )



In [9]:
from model_server.srv import ImageRequest, ImageRequestResponse

In [20]:
var = ImageRequest

In [24]:
ImageRequestResponse()

status: 
  data: False

In [25]:
from model_server.msg import InferenceResults

In [32]:
print(InferenceResults())
print (f'Structure Datatype: {type(InferenceResults().structure)}')
print (f'Inferences Datatype: {type(InferenceResults().inferences)}')

structure: []
inferences: []
Structure Datatype: <class 'list'>
Inferences Datatype: <class 'list'>
